In [ ]:
# 3RD VERSION: Using sklearn.linear_model.LinearRegression to standardize the regression analysis with HDB

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px
import re


# STEP 1: PREPARE DATA
df_private = pd.read_csv('../source_data/SOURCE-Private-D01,02,03,04,05,07,09,10,11,12,13,14,15,19,20,21-Transaction202403.csv')

# Rename columns to avoid issues with special characters and spaces
df_private.rename(columns={
    'Unit Price ($ PSF)': 'Unit_Price_PSF',
    'Transacted Price ($)': 'Transacted_Price',
    'Area (SQFT)': 'Area_SQFT',
    'Sale Date': 'Sale_Date',
    'Floor Level': 'Floor_Level',
    'Project Name': 'Project_Name'
}, inplace=True)

# Clean the Data
df_private['Sale_Date'] = pd.to_datetime(df_private['Sale_Date'], format='%b-%y')
df_private['Unit_Price_PSF'] = df_private['Unit_Price_PSF'].str.replace(',', '').astype(int)
df_private['Transacted_Price'] = df_private['Transacted_Price'].str.replace(',', '').astype(int)
df_private['Area_SQFT'] = df_private['Area_SQFT'].str.replace(',', '').astype(float)

common_start_date = pd.Timestamp('2019-03-01')

# Calculate months since common start for each entry
df_private['months_since_common_start'] = ((df_private['Sale_Date'] - common_start_date) / np.timedelta64(1, 'M')).astype(int)

# print(f"df_private {df_private}")
# print(df_private.dtypes)


gradients = []


def parse_tenure(tenure):
    lease_type_pattern = r"^(Freehold|\d+\s+yrs\s+lease)"
    commence_year_pattern = r"(\d{4})$"

    lease_type_match = re.search(lease_type_pattern, tenure)
    commence_year_match = re.search(commence_year_pattern, tenure)

    # Extract matched values or default to None if not found
    lease_type = lease_type_match.group(0) if lease_type_match else None
    commence_year = commence_year_match.group(0) if commence_year_match else None

    return lease_type, commence_year


# Iterate over each unique project name in the private housing dataset
for project_name in df_private['Project_Name'].unique():
    project_data = df_private[df_private['Project_Name'] == project_name].copy()

    if project_data.empty:
        continue

    num_data_points = len(project_data)
    if num_data_points < 4:
      continue

    # Use months_since_common_start as the independent variable for regression
    X = project_data[['months_since_common_start']].values
    y = project_data['Unit_Price_PSF'].values

    model = LinearRegression().fit(X, y)

    start_date = project_data['Sale_Date'].min()
    end_date = project_data['Sale_Date'].max()
    months_between = round((end_date - start_date) / np.timedelta64(1, 'M'))

    predicted_psf_start = model.predict(np.array([[project_data['months_since_common_start'].min()]]))[0]
    predicted_psf_end = model.predict(np.array([[project_data['months_since_common_start'].max()]]))[0]

    base_price = predicted_psf_start

    monthly_gradient = model.coef_[0]
    gradient_per_year = monthly_gradient * 12
    gradient_percentage = (gradient_per_year / base_price) * 100
    postal_district = project_data['Postal District'].mode()[0]
    lease_type, top_year = parse_tenure(project_data['Tenure'].mode()[0])

    gradients.append((postal_district, project_name, top_year, lease_type, gradient_per_year, gradient_percentage, predicted_psf_start, predicted_psf_end, start_date, end_date, months_between, num_data_points))

    print(f"Gradient for {project_name}: {gradient_per_year:.4f} per year, {gradient_percentage:.2f}%")
    print(f"Postal District {postal_district}. Lease Type: {lease_type}, TOPed in {top_year}")
    print(f"Predicted PSF at start: ${predicted_psf_start:.2f}, at end: ${predicted_psf_end:.2f}")
    print(f"From {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}, over {months_between} months. {num_data_points} data points")



gradients_df = pd.DataFrame(gradients, columns=['Postal District', 'Project Name', 'TOP Year', 'Lease Type', 'Gradient ($PSF per year)', 'Gradient % ($PSF per year)', 'Predicted PSF Start', 'Predicted PSF End', 'Start Date', 'End Date', 'Months Between', 'No of Data Points'])
print(gradients_df)
gradients_df.to_csv('gradients_df.csv', index=False)
from google.colab import files
files.download('gradients_df.csv')



Streaming output truncated to the last 5000 lines.
Postal District 11. Lease Type: Freehold, TOPed in None
Predicted PSF at start: $1485.04, at end: $1523.19
From 2020-10 to 2021-10, over 12 months. 11 data points
Gradient for TAN TONG MENG TOWER: 62.3030 per year, 6.38%
Postal District 11. Lease Type: Freehold, TOPed in None
Predicted PSF at start: $977.30, at end: $1117.48
From 2019-05 to 2021-08, over 27 months. 5 data points
Gradient for PASADENA: -175.0242 per year, -13.85%
Postal District 11. Lease Type: Freehold, TOPed in None
Predicted PSF at start: $1263.58, at end: $928.11
From 2019-09 to 2021-08, over 23 months. 5 data points
Gradient for TEN @ SUFFOLK: 43.1777 per year, 3.03%
Postal District 11. Lease Type: Freehold, TOPed in None
Predicted PSF at start: $1426.82, at end: $1516.77
From 2019-05 to 2021-06, over 25 months. 7 data points
Gradient for TREVISTA: 85.7437 per year, 6.57%
Postal District 12. Lease Type: 99 yrs lease, TOPed in 2008
Predicted PSF at start: $1305.46, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Display graph only for selected projects

import plotly.express as px
import plotly.graph_objects as go
from google.colab import files

projects_to_plot = [
    # DISTRICT 01 / 02 CENTRAL
    'EMERALD GARDEN',
    'SPOTTISWOODE SUITES',
    'SPOTTISWOODE RESIDENCES',
    'CRAIG PLACE',
    'THE ARRIS',
    'THE BEACON',
    'SKYSUITES@ANSON',
    'THE SAIL @ MARINA BAY',
    'MARINA BAY RESIDENCES',
    'V ON SHENTON',
    'EON SHENTON',

    # DISTRICT 11
    'CUBE 8',
    'THE ARCADIA',
]

for project_name in projects_to_plot:
    project_data = df_private[df_private['Project_Name'] == project_name].copy()

    if project_data.empty:
        print(f"No data available for {project_name}.")
        continue

    # Perform regression to get the line of best fit
    X = project_data[['months_since_common_start']].values
    y = project_data['Unit_Price_PSF'].values
    model = LinearRegression().fit(X, y)
    project_data['predicted_psf'] = model.predict(X)


    # Plotly way consistent with HDB START
    hover_text = ['Date: {}<br>Unit Price PSF: ${:.2f}'.format(date.strftime('%Y-%m-%d'), psf)
              for date, psf in zip(project_data['Sale_Date'], project_data['predicted_psf'])]

    fig = px.scatter(project_data, x='Sale_Date', y='Unit_Price_PSF',
                    hover_data=['Transacted_Price', 'Area_SQFT', 'Floor_Level'],
                    labels={'Unit_Price_PSF': 'Unit Price (PSF)', 'Sale_Date': 'Time'},
                    title=f"Unit Price (PSF) Over Time for {project_name}")


    fig.add_trace(go.Scatter(x=project_data['Sale_Date'], y=project_data['predicted_psf'],
                            mode='lines', name='Regression Line',
                            hoverinfo='text',
                            text=hover_text))



    # Plotly way consistent with HDB END

# NEW GO WAY START
    # # Create the plot
    # fig = go.Figure()

    # # Add scatter plot for actual data points
    # fig.add_trace(go.Scatter(x=project_data['Sale_Date'], y=project_data['Unit_Price_PSF'], mode='markers',
    #                          name='Actual Data',
    #                          hoverinfo='text',
    #                          text=project_data.apply(lambda row: f"Transaction Price: {row['Transacted_Price']}<br>"
    #                                                               f"Floor Area: {row['Area_SQFT']} sqft<br>"
    #                                                               f"Storey Range: {row['Floor_Level']}<br>"
    #                                                               f"Unit Price PSF: {row['Unit_Price_PSF']}", axis=1)))

    # # Add line plot for regression line
    # fig.add_trace(go.Scatter(x=project_data['Sale_Date'], y=project_data['predicted_psf'], mode='lines',
    #                          name='Regression Line'))

    # # Update plot layout
    # fig.update_layout(title=f"Unit Price PSF over Time for {project_name}",
    #                   xaxis_title='Sale Date',
    #                   yaxis_title='Unit Price PSF',
    #                   hovermode='closest')
# NEW GO WAY END

    fig.show()

    project_data = project_data.sort_values(by='Sale_Date')
    base_period = project_data[project_data['Sale_Date'].dt.year == project_data['Sale_Date'].dt.year.min()]
    base_price = base_period['Unit_Price_PSF'].mean()

    start_date = project_data['Sale_Date'].min()
    end_date = project_data['Sale_Date'].max()
    months_between = round((end_date - start_date) / np.timedelta64(1, 'M'))

    predicted_psf_start = model.predict(np.array([[project_data['months_since_common_start'].min()]]))[0]
    predicted_psf_end = model.predict(np.array([[project_data['months_since_common_start'].max()]]))[0]

    # Calculate the gradient of Unit Price (PSF) over time
    monthly_gradient = model.coef_[0]
    gradient_per_year = monthly_gradient * 12
    gradient_percentage = (gradient_per_year / base_price) * 100
    num_data_points = len(project_data)

    print(f"Gradient for {project_name}: gradient_per_year {gradient_per_year:.4f}, gradient_percentage {gradient_percentage:.2f}%")
    print(f"Predicted PSF at start: {predicted_psf_start:.2f}, at end: {predicted_psf_end:.2f}, from {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}, over {months_between} months. {num_data_points} data points")

    file_name = f"{project_name.replace(' ', '_')}_Price_Trend.html"  # Replace spaces with underscores
    fig.write_html(file_name)
    files.download(file_name)


Gradient for EMERALD GARDEN: gradient_per_year 94.3028, gradient_percentage 4.85%
Predicted PSF at start: 1870.63, at end: 2263.56, from 2019-12 to 2024-02, over 49 months. 21 data points


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Gradient for SPOTTISWOODE SUITES: gradient_per_year 102.4717, gradient_percentage 4.83%
Predicted PSF at start: 1951.49, at end: 2395.54, from 2019-10 to 2024-03, over 52 months. 34 data points


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Gradient for THE ARCADIA: gradient_per_year 72.4048, gradient_percentage 7.60%
Predicted PSF at start: 869.05, at end: 1212.98, from 2019-04 to 2024-02, over 57 months. 46 data points


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>